<br>

# <center> Qiana - Hyper Parameter Tuning using GridSearchCV

<br>

---

<br>


# List of Contents



*   [1. Initialization](#initialization)
*   [2. NLP - Text Processing](#nlp)
*   [3. Preprocessing](#preprocessing)
*   [4. Model Design](#model_design)
*   [5. Hyper Parameter Tuning](#hyper_tuning)


<br>


<br>
<br>

<a name='initialization'></a>
# 1. Initialization

<br>

## 1.1. Colab Configuration

### 1.1.1. Mount Google Drive

In [ ]:
'''
    This is required if the code runs in Google Colab.
    - this code will mount Google Drive for Colab.
    - the code needs to run only once.
'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<br>

### 1.1.2. Defining Root Directory

In [ ]:
# this directory will be used as Root Directory to read/write any file
rootDir = '/content/drive/MyDrive/_ML/Qiana/'

<br>

## 1.2. Import Libraries

In [ ]:
# importing all the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
import json

<br>

## 1.3. Import DataSet

<br> 

> Reading JSON DataSet

In [ ]:
# reading JSON file
filePath = rootDir + '02. DataSet/intents.json'
with open(filePath,'r') as data: 
  dataset = json.load(data)

In [ ]:
# converting to Dictionary
dataset = dict(dataset)

dataset = dataset['intents']

<br>

> Defining Lists

In [ ]:
messages = []  # to store message
msgIntents = []  # to store intents of messages
intents = []  # to store intent

responses = {}  # to store responses for each intents

responseType = {}  # to store response type for each intents

<br>

> Extract Messages, Intents, Responses, Response Type from Data

In [ ]:
for item in dataset:
  intent = item['intent']
  texts = item['texts']
  res = item['reply']
  res_type = item['response_type']

  for text in texts:
    messages.append(text)
    msgIntents.append(intent)  

  if intent not in intents:
    intents.append(intent)

  responses[intent] = res 

  responseType[intent] = res_type    

In [ ]:
intents = sorted(intents)

<br>
<br>

<a name='nlp'></a>

# 2. NLP - Text Processing

<br>

## 2.1. Initializing NLTK

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

<br>

## 2.2. Tokenization

In [ ]:
def _tokenize(words):
  from nltk.tokenize import sent_tokenize, word_tokenize

  _tokenized_words = []

  for word in words:
    word_token = word_tokenize(word)
    _tokenized_words.append(word_token)

  return _tokenized_words

<br>

## 2.3. Filtering Stop Words

In [ ]:
def _filter_stopword(word_list):
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words("english"))
  
  _filtered_words = []

  for words in word_list:
    tempList = []
    for word in words:
      if word.casefold() not in stop_words:
        tempList.append(word)
    _filtered_words.append(tempList)
  
  return _filtered_words

<br>

## 2.4. Stemming

In [ ]:
def _stem(word_list):
  from nltk.stem import PorterStemmer
  stemmer = PorterStemmer()

  _stemmed_words = []

  for words in word_list:
    tempList = [stemmer.stem(word) for word in words]
    _stemmed_words.append(tempList)

  return _stemmed_words

<br>

## 2.5. Parts of Speech Tagging 

In [ ]:
def _pos_tag(word_list):
  _pos_tagged_words = []

  for words in word_list:
    tagged_words = nltk.pos_tag(words)
    _pos_tagged_words.append(tagged_words)

  return _pos_tagged_words

<br>

## 2.6. Lemmatizing

In [ ]:
def _lemmatize(word_list):
  from nltk.corpus import wordnet
  def get_wordnet_pos(word):
      """Map POS tag to first character lemmatize() accepts"""
      
      tag = nltk.pos_tag([word])[0][1][0].lower()
      
      tag_dict = {"a": wordnet.ADJ,
                  "n": wordnet.NOUN,
                  "v": wordnet.VERB,
                  "r": wordnet.ADV}
      
      return tag_dict.get(tag, wordnet.NOUN)

  from nltk.stem import WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()

  _lemmatized_words = []

  for words in word_list:
    tempList = []
    
    for word in words:
      pos = get_wordnet_pos(word)
      lemma = lemmatizer.lemmatize(word=word, pos=pos)
      tempList.append(lemma.lower())
    
    _lemmatized_words.append(tempList)

  return _lemmatized_words

<br>

## 2.7. Generate Bag of Words

In [ ]:
def _generate_bag_of_words(word_list):
  tempBag = []

  for words in word_list:
    tempBag.extend(words)

  tempBag = set(tempBag)

  _bag_of_words = list( tempBag )

  _bag_of_words = sorted(_bag_of_words)

  return _bag_of_words

<br>

## 2.8. Data Matrix Generator

In [ ]:
def matrix_generator(word_list):
  template_row = [0 for _ in range(0, len(bag_of_words))]

  data_matrix = []

  for words in word_list:
    mat_row = template_row[:]

    for word in words:
      if word in bag_of_words:
        word_index = bag_of_words.index(word)
        _count = words.count(word)
        mat_row[word_index] = _count
    
    data_matrix.append(mat_row)
  
  return data_matrix 

<br>

## 2.9. Targets Generator

In [ ]:
def generate_targets():
  targets = []

  for intent in msgIntents:
    index = intents.index(intent)
    targets.append(index)

  # coverting to Categorical
  from tensorflow.keras.utils import to_categorical
  targets = to_categorical(targets)

  return targets

<br>

## 2.10. Text Processing

In [ ]:
# tokenizing
tokenized_words = _tokenize(messages)

In [ ]:
# stemmimg
stemmed_words = _stem(tokenized_words)

In [ ]:
# pos tagging
pos_tagged_words = _pos_tag(tokenized_words)

In [ ]:
# lemmatizing
lemmatized_words = _lemmatize(tokenized_words)

In [ ]:
# generating bag of words
bag_of_words = _generate_bag_of_words(lemmatized_words)

In [ ]:
len(bag_of_words)

186

In [ ]:
# generating matrix
data_matrix = matrix_generator(lemmatized_words)

In [ ]:
# creating dataframe
df = pd.DataFrame(data=data_matrix, columns=bag_of_words)

In [ ]:
df.head()

,!,'m,'s,",",.,;,?,a,about,adios,...,what,which,who,why,with,witty,would,you,your,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# generating targets
targets = generate_targets()

<br>
<br>

<a name='preprocessing'></a>

# 3. Preprocessing

In [ ]:
# Train Test Split
X = df.values
y = targets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

<br>
<br>

<a name='model_design'></a>

# 4. Model Design

In [ ]:
# importing libraries for model design
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [ ]:
# checking the shape of the training data
X_train.shape

(299, 183)

<br>
<br>

<a name='hyper_tuning'></a>

# 5. Hyper Parameter Tuning

<br>

## 5.1. Grid Search CV - batch_size & epochs

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model ():

    model = Sequential()

    model.add( Dense(units=183, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=115, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=230, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=176, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=88, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=44, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    # Compiling the model
    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer='adam'
    )

    return model

In [ ]:
# define the grid search parameters
batch_size = [10, 20, 30, 40, 50, 60, 70]
epochs = [110, 120]

param_grid = {
    'batch_size' : batch_size,
    'epochs': epochs
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
grid_result = grid.fit(X, y)

# summarize grid search information
bestScore = grid_result.best_score_
bestParams = grid_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

Fitting 3 folds for each of 14 candidates, totalling 42 fits


<ipython-input-43-05ac00a303bd>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


[Result] Best score is 0.94 using {'batch_size': 20, 'epochs': 120}


<br>

## 5.2. Grid Search CV - Optimizer

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model (optimizer):

    model = Sequential()

    model.add( Dense(units=183, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=115, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=230, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=176, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=88, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=44, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    # Compiling the model
    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer=optimizer
    )

    return model

In [ ]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = {
    'optimizer' : optimizer
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, optimizer='adam', batch_size=20, epochs= 120, verbose=0)

<ipython-input-74-d5b538053d78>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, optimizer='adam', batch_size=20, epochs= 120, verbose=0)


In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
optimizer_search_result = grid.fit(X, y)

# summarize grid search information
bestScore = optimizer_search_result.best_score_
bestParams = optimizer_search_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

# extract the best model, make predictions on our data, and show a
# classification report
# print("[INFO] evaluating the best model...")
# bestModel = grid_result.best_estimator_
# accuracy = bestModel.score(X_test, y_test)
# print("accuracy: {:.2f}%".format(accuracy * 100))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
[Result] Best score is 0.92 using {'optimizer': 'RMSprop'}


<br>

## 5.3. Grid Search CV - Learning Rate

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model (learning_rate):

    model = Sequential()

    model.add( Dense(units=183, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=115, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=230, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=176, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=88, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=44, activation='tanh') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    # Compiling the model
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer=optimizer
    )

    return model

In [ ]:
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = {
    'learning_rate' : learning_rate
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, learning_rate = 0.01, batch_size=20, epochs= 120, verbose=0)

<ipython-input-105-38e4c0400880>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, learning_rate = 0.01, batch_size=20, epochs= 120, verbose=0)


In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
learnRate_search_result = grid.fit(X, y)

# summarize grid search information
bestScore = learnRate_search_result.best_score_
bestParams = learnRate_search_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

# extract the best model, make predictions on our data, and show a
# classification report
# print("[INFO] evaluating the best model...")
# bestModel = grid_result.best_estimator_
# accuracy = bestModel.score(X_test, y_test)
# print("accuracy: {:.2f}%".format(accuracy * 100))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[Result] Best score is 0.92 using {'learning_rate': 0.001}


<br>

## 5.4. Grid Search CV - activation

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model (activation1, activation2):

    model = Sequential()

    model.add( Dense(units=183, activation=activation1) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=115, activation=activation2) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=230, activation=activation1) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=176, activation=activation2) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=88, activation=activation1) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=44, activation=activation2) )
    model.add( Dropout(0.4) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    
    # Compiling the model
    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer='RMSprop' # default learning rate is 0.001
    )

    return model

In [ ]:
activation= ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = {
    'activation1' : activation,
    'activation2' : activation,
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)

<ipython-input-118-523ffa6f36dd>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)


In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
activation_search_result = grid.fit(X, y)

# summarize grid search information
bestScore = activation_search_result.best_score_
bestParams = activation_search_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

# extract the best model, make predictions on our data, and show a
# classification report
# print("[INFO] evaluating the best model...")
# bestModel = grid_result.best_estimator_
# accuracy = bestModel.score(X_test, y_test)
# print("accuracy: {:.2f}%".format(accuracy * 100))

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[Result] Best score is 0.92 using {'activation1': 'tanh', 'activation2': 'softsign'}


<br>

## 5.5. Grid Search CV - drop_out

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model (dropout1, dropout2, dropout3, dropout4, dropout5, dropout6):

    model = Sequential()

    model.add( Dense(units=183, activation='tanh') )
    model.add( Dropout(dropout1) )

    model.add( Dense(units=115, activation='tanh') )
    model.add( Dropout(dropout2) )

    model.add( Dense(units=230, activation='tanh') )
    model.add( Dropout(dropout3) )

    model.add( Dense(units=176, activation='tanh') )
    model.add( Dropout(dropout4) )

    model.add( Dense(units=88, activation='tanh') )
    model.add( Dropout(dropout5) )

    model.add( Dense(units=44, activation='tanh') )
    model.add( Dropout(dropout6) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    
    # Compiling the model
    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer='RMSprop' # default learning rate is 0.001
    )

    return model

In [ ]:
dropout = [0.3, 0.4]
param_grid = {
    'dropout1' : [0.3, 0.4],
    'dropout2' : [0.4, 0.5],
    'dropout3' : [0.25, 0.4],
    'dropout4' : [0.4, 0.6],
    'dropout5' : [0.1, 0.4],
    'dropout6' : [0.3, 0.4]
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)

<ipython-input-124-523ffa6f36dd>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)


In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
dropOut_search_result = grid.fit(X, y)

# summarize grid search information
bestScore = dropOut_search_result.best_score_
bestParams = dropOut_search_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

# extract the best model, make predictions on our data, and show a
# classification report
# print("[INFO] evaluating the best model...")
# bestModel = grid_result.best_estimator_
# accuracy = bestModel.score(X_test, y_test)
# print("accuracy: {:.2f}%".format(accuracy * 100))

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[Result] Best score is 0.94 using {'dropout1': 0.3, 'dropout2': 0.4, 'dropout3': 0.25, 'dropout4': 0.6, 'dropout5': 0.1, 'dropout6': 0.4}


<br>

## 5.6. Grid Search CV - Neuron

In [ ]:
X = X_train
y = y_train

In [ ]:
# model creatiion

def create_model (neurons1, neurons2, neurons3, neurons4, neurons5):

    model = Sequential()

    model.add( Dense(units=183, activation='tanh') )
    model.add( Dropout(0.3) )

    model.add( Dense(units=neurons1, activation='softsign') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=neurons2, activation='tanh') )
    model.add( Dropout(0.25) )

    model.add( Dense(units=neurons3, activation='softsign') )
    model.add( Dropout(0.6) )

    model.add( Dense(units=neurons4, activation='tanh') )
    model.add( Dropout(0.1) )

    model.add( Dense(units=neurons5, activation='softsign') )
    model.add( Dropout(0.4) )

    model.add( Dense(units=22, activation = 'softmax') )
    
    
    # Compiling the model
    model.compile(
        loss='categorical_crossentropy',
        metrics = ['accuracy'],
        optimizer='RMSprop' # default learning rate is 0.001
    )

    return model

In [ ]:
param_grid = {
    'neurons1' : [135, 140, 145],
    'neurons2' : [245, 250, 255],
    'neurons3' : [150, 155, 160],
    'neurons4' : [ 85,  88,  90],
    'neurons5' : [175, 180, 185],
}

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)

<ipython-input-158-523ffa6f36dd>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, batch_size=20, epochs= 120, verbose=0)


In [ ]:
# importing Library
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
neuron_search_result = grid.fit(X, y)

# summarize grid search information
bestScore = neuron_search_result.best_score_
bestParams = neuron_search_result.best_params_
print("[Result] Best score is {:.2f} using {}".format(bestScore, bestParams))

# extract the best model, make predictions on our data, and show a
# classification report
# print("[INFO] evaluating the best model...")
# bestModel = grid_result.best_estimator_
# accuracy = bestModel.score(X_test, y_test)
# print("accuracy: {:.2f}%".format(accuracy * 100))

Fitting 3 folds for each of 243 candidates, totalling 729 fits
